# 📚 מדריך מקיף: קריאה וכתיבה של קבצים ב-Polars

**ברוכים הבאים למדריך המקיף לעבודה עם קבצים ב-Polars!**

מדריך זה מכסה את כל פורמטי הקבצים הנפוצים ומספק הסברים מפורטים, דוגמאות קוד, ותרגילים אינטראקטיביים.

---

## 📑 תוכן עניינים

1. [קריאה וכתיבה של קבצי CSV](#1-קריאה-וכתיבה-של-קבצי-csv)
2. [קריאה וכתיבה של קבצי Parquet](#2-קריאה-וכתיבה-של-קבצי-parquet)
3. [קריאה וכתיבה של טבלאות Delta Lake](#3-קריאה-וכתיבה-של-טבלאות-delta-lake)
4. [קריאה וכתיבה של קבצי JSON](#4-קריאה-וכתיבה-של-קבצי-json)
5. [קריאה וכתיבה של קבצי Excel](#5-קריאה-וכתיבה-של-קבצי-excel)
6. [פורמטים נוספים (IPC, Avro, Iceberg)](#6-פורמטים-נוספים)
7. [עבודה עם קבצים מרובים](#7-עבודה-עם-קבצים-מרובים)
8. [עבודה עם בסיסי נתונים](#8-עבודה-עם-בסיסי-נתונים)
9. [סיכום וטיפים](#9-סיכום-וטיפים)

---

## הכנה ראשונית

### התקנת Polars

קודם כל, נוודא ש-Polars מותקן:

In [ ]:
# התקנת Polars (הרץ רק פעם אחת)
# !pip install polars

In [ ]:
# ייבוא הספרייה
import polars as pl

# בדיקת גרסה
print(f"Polars version: {pl.__version__}")

---

## 1. קריאה וכתיבה של קבצי CSV

### 📖 מה זה CSV?

**CSV (Comma-Separated Values)** הוא פורמט קובץ טקסט פשוט המשמש לאחסון נתונים טבלאיים. כל שורה מייצגת רשומה, וכל ערך מופרד בפסיק (או תו אחר).

**יתרונות:**
- 📝 קל לקריאה ועריכה
- 🔄 תואם למגוון כלים
- 💾 קובץ טקסט פשוט

**חסרונות:**
- 📏 אין שמירת טיפוסי נתונים
- 🐌 איטי יחסית לקבצים גדולים
- 💾 גודל קובץ גדול יחסית

---

### 🔍 קריאה בסיסית של CSV

הדרך הפשוטה ביותר לקרוא קובץ CSV:

In [ ]:
# קריאה בסיסית של קובץ CSV
df = pl.read_csv('../data/customer_shopping_data.csv')

# הצגת 5 השורות הראשונות
df.head()

#### 💡 הסבר:

- `pl.read_csv()` - הפונקציה העיקרית לקריאת CSV
- `df.head()` - מציג את 5 השורות הראשונות (ברירת מחדל)
- Polars מזהה אוטומטית את טיפוסי הנתונים בעמודות

### 🚫 עבודה עם קבצים ללא כותרות (Headers)

לפעמים קבצי CSV לא כוללים שורת כותרות. בואו נראה איך מטפלים במצב זה:

In [ ]:
# ניסיון ראשון - Polars חושב שהשורה הראשונה היא כותרת
df_wrong = pl.read_csv('../data/customer_shopping_data_no_header.csv')
print("❌ בעייתי - השורה הראשונה הפכה לכותרת:")
df_wrong.head()

In [ ]:
# ✅ הדרך הנכונה - אומרים ל-Polars שאין כותרות
df_correct = pl.read_csv(
    '../data/customer_shopping_data_no_header.csv',
    has_header=False  # 👈 זה הפרמטר החשוב!
)
print("✅ נכון - Polars יצר כותרות אוטומטיות (column_1, column_2...):")
df_correct.head()

#### 💡 הסבר:

- `has_header=False` - מודיע ל-Polars שאין שורת כותרות
- Polars יוצר כותרות אוטומטיות: `column_1`, `column_2`, וכו'

### 🏷️ הגדרת שמות עמודות מותאמים אישית

במקום לקבל שמות עמודות גנריים, נוכל להגדיר שמות משלנו:

In [ ]:
# הגדרת רשימת שמות עמודות
column_names = [
    'invoice_no',      # מספר חשבונית
    'customer_id',     # מזהה לקוח
    'gender',          # מין
    'age',             # גיל
    'category',        # קטגוריה
    'quantity',        # כמות
    'price',           # מחיר
    'payment_method',  # אמצעי תשלום
    'invoice_date',    # תאריך חשבונית
    'shopping_mall'    # קניון
]

# קריאה עם שמות עמודות מותאמים
df_named = pl.read_csv(
    '../data/customer_shopping_data_no_header.csv',
    has_header=False,
    new_columns=column_names  # 👈 רשימת השמות שלנו
)

df_named.head()

#### 💡 הסבר:

- `new_columns` - מקבל רשימה של שמות עמודות
- מספר השמות חייב להתאים למספר העמודות בקובץ
- שמות העמודות יכולים להיות בכל שפה (כולל עברית!)

### 📅 ניתוח אוטומטי של תאריכים

Polars יכול לזהות ולהמיר תאריכים אוטומטית:

In [ ]:
# קריאה עם ניתוח תאריכים
df_with_dates = pl.read_csv(
    '../data/customer_shopping_data_no_header.csv',
    has_header=False,
    new_columns=column_names,
    try_parse_dates=True  # 👈 מנסה לזהות תאריכים אוטומטית
)

print("📋 סכמת הנתונים (Data Schema):")
print(df_with_dates.schema)
print("\n📊 הנתונים:")
df_with_dates.head()

#### 💡 שימו לב:

- עמודת `invoice_date` השתנתה מ-`str` ל-`date`
- עכשיו ניתן לבצע פעולות של תאריכים (חישוב הפרשים, סינון לפי טווח, וכו')

### 🎯 שליטה מדויקת בטיפוסי נתונים

לפעמים נרצה לשלוט בדיוק על טיפוס כל עמודה (לחסוך זיכרון או למנוע שגיאות):

In [ ]:
# בדיקה: איזה טיפוס לעמודת 'age' כרגע?
print(f"טיפוס age לפני: {df_with_dates['age'].dtype}")

# קריאה עם הגדרת טיפוסים מותאמים
df_optimized = pl.read_csv(
    '../data/customer_shopping_data_no_header.csv',
    has_header=False,
    new_columns=column_names,
    try_parse_dates=True,
    schema_overrides={
        'age': pl.Int8,        # 👈 Int8 = -128 עד 127 (חוסך זיכרון!)
        'quantity': pl.Int32   # 👈 Int32 = מספרים שלמים רגילים
    }
)

print(f"טיפוס age אחרי: {df_optimized['age'].dtype}")
print("\n📋 סכמה מלאה:")
print(df_optimized.schema)

#### 💡 מדוע זה חשוב?

- **Int64** (ברירת מחדל): 8 בתים לכל ערך
- **Int8**: רק 1 בייט לכל ערך! (חיסכון של פי 8)
- עבור מיליוני שורות, ההבדל משמעותי

**טיפוסי מספרים נפוצים:**
- `Int8`: -128 עד 127
- `Int16`: -32,768 עד 32,767
- `Int32`: -2.1B עד 2.1B
- `Int64`: מספרים ענקיים

### 💾 כתיבת קבצי CSV

עכשיו נלמד איך לשמור DataFrame כקובץ CSV:

In [ ]:
# שמירה בסיסית
df_optimized.write_csv('../data/output/shopping_data_output.csv')
print("✅ הקובץ נשמר בהצלחה!")

In [ ]:
# שמירה מתקדמת - בלי כותרות, עם מפריד אחר
df_optimized.write_csv(
    '../data/output/shopping_data_custom.csv',
    include_header=False,  # ללא שורת כותרות
    separator='|'          # מפריד: | במקום ,
)
print("✅ נשמר עם הגדרות מותאמות!")

#### ⚙️ פרמטרים שימושיים נוספים:

```python
df.write_csv(
    path='output.csv',
    include_header=True,     # האם לכלול כותרות
    separator=',',           # תו מפריד
    quote_char='"',          # תו ציטוט
    datetime_format='%Y-%m-%d'  # פורמט תאריכים
)
```

### ⚡ Lazy Loading - עיבוד יעיל של קבצים ענקיים

**מה זה Lazy Loading?**

במקום לטעון את כל הקובץ לזיכרון, Polars:
1. 📋 קורא רק את המטא-דאטה
2. 🎯 מבין אילו חישובים דרושים
3. ⚡ מריץ רק את מה שצריך
4. 💪 יכול לעבד קבצים גדולים מהזיכרון!

In [ ]:
# יצירת LazyFrame (לא טוען את הנתונים!)
lf = pl.scan_csv(
    '../data/customer_shopping_data_no_header.csv',
    has_header=False,
    new_columns=column_names,
    try_parse_dates=True,
    schema_overrides={'age': pl.Int8, 'quantity': pl.Int32}
)

print("🔍 זה LazyFrame - עדיין לא טענו נתונים!")
print(type(lf))

# כדי לראות נתונים, צריך לקרוא ל-collect()
result = lf.head().collect()
print("\n📊 עכשיו הנתונים נטענו:")
result

#### 💪 דוגמה לעוצמה של Lazy:

נניח שיש לנו קובץ ענק, ואנחנו רוצים רק 10 שורות מקטגוריה מסוימת:

In [ ]:
# Lazy - יקרא רק את מה שצריך!
result = (
    pl.scan_csv('../data/customer_shopping_data_no_header.csv',
                has_header=False, new_columns=column_names)
    .filter(pl.col('category') == 'Clothing')  # סינון
    .select(['customer_id', 'price', 'quantity'])  # בחירת עמודות
    .head(10)  # רק 10 שורות
    .collect()  # רק עכשיו מריץ!
)

print("⚡ Polars קרא רק את מה שדרוש - מהיר מאוד!")
result

### 📤 Streaming - כתיבה ישירה לדיסק

`sink_csv()` שומר ישירות לקובץ **מבלי לטעון הכל לזיכרון**!

In [ ]:
# עיבוד והשמרה ב-streaming mode
(
    pl.scan_csv('../data/customer_shopping_data_no_header.csv',
                has_header=False, new_columns=column_names)
    .filter(pl.col('age') > 30)
    .sink_csv('../data/output/adults_only.csv')  # 👈 ישירות לדיסק!
)

print("✅ הקובץ נשמר בצורה אופטימלית!")

---

### 🎓 תרגיל 1: קריאת CSV

**משימה:** קרא את הקובץ `customer_shopping_data.csv` ו:
1. המר את עמודת התאריכים לטיפוס `date`
2. שנה את `age` ל-`Int8` ו-`quantity` ל-`Int16`
3. סנן רק רכישות מעל 1000 ש"ח
4. הצג את 5 השורות הראשונות

In [ ]:
# 💪 נסה בעצמך כאן:

# df = pl.read_csv(...)
# ...
# df.head()

<details>
<summary>💡 פתרון (לחץ להצגה)</summary>

```python
df_exercise = pl.read_csv(
    '../data/customer_shopping_data.csv',
    try_parse_dates=True,
    schema_overrides={
        'age': pl.Int8,
        'quantity': pl.Int16
    }
).filter(pl.col('price') > 1000)

df_exercise.head()
```
</details>

---

### 📚 סיכום: CSV

| פעולה | פונקציה | שימוש |
|--------|---------|-------|
| קריאה רגילה | `pl.read_csv()` | קבצים קטנים-בינוניים |
| קריאה lazy | `pl.scan_csv()` | קבצים גדולים, אופטימיזציה |
| כתיבה רגילה | `df.write_csv()` | שמירה סטנדרטית |
| כתיבה streaming | `lf.sink_csv()` | קבצים ענקיים |

**טיפים מרכזיים:**
- ✅ השתמש ב-`try_parse_dates=True` לתאריכים
- ✅ אופטימיזציה: `schema_overrides` לחיסכון בזיכרון
- ✅ קבצים גדולים: `scan_csv()` + `collect()`
- ✅ קבצים ענקיים: `scan_csv()` + `sink_csv()`

---

## 2. קריאה וכתיבה של קבצי Parquet

### 📖 מה זה Parquet?

**Apache Parquet** הוא פורמט קובץ **עמודתי** (columnar) המאוחסן בצורה **דחוסה** ו**יעילה**.

**למה Parquet עדיף על CSV?**

| תכונה | CSV | Parquet |
|--------|-----|----------|
| גודל קובץ | 📊 גדול | 📉 קטן (דחוס) |
| מהירות קריאה | 🐌 איטי | ⚡ מהיר מאוד |
| שמירת טיפוסים | ❌ לא | ✅ כן |
| קריאה חלקית | ❌ קורא הכל | ✅ רק עמודות נדרשות |

**מתי להשתמש ב-Parquet?**
- 📊 אחסון ארוך טווח
- ⚡ צורך בביצועים גבוהים
- 💾 קבצים גדולים
- 🔄 עבודה עם נתונים בענן (S3, Azure, GCS)

### 🔍 קריאה בסיסית של Parquet

In [ ]:
# קריאה פשוטה
df_parquet = pl.read_parquet('../data/venture_funding_deals.parquet')

print("📋 מידע על ה-DataFrame:")
print(f"מספר שורות: {len(df_parquet):,}")
print(f"מספר עמודות: {len(df_parquet.columns)}")
print(f"\nשמות עמודות: {df_parquet.columns}")

df_parquet.head()

### 🎯 קריאת עמודות ספציפיות בלבד

**אחד היתרונות הגדולים של Parquet**: ניתן לקרוא רק חלק מהעמודות!

In [ ]:
# קריאת עמודות ספציפיות בלבד
df_selected = pl.read_parquet(
    '../data/venture_funding_deals.parquet',
    columns=['Company', 'Amount', 'Valuation', 'Industry']  # 👈 רק אלה!
)

print("⚡ נקראו רק 4 עמודות - חסכנו זמן וזיכרון!")
df_selected.head()

#### 💡 למה זה מהיר?

Parquet מאוחסן בצורה עמודתית:
```
CSV (שורתי):           Parquet (עמודתי):
row1: A, B, C         col_A: [A, A, A, ...]
row2: A, B, C         col_B: [B, B, B, ...]
row3: A, B, C         col_C: [C, C, C, ...]
```

כדי לקרוא עמודה אחת, Parquet פשוט קופץ למיקום שלה!

### 📋 קריאת מטא-דאטה בלבד (ללא הנתונים)

לפעמים נרצה לדעת מה יש בקובץ בלי לטעון אותו:

In [ ]:
# קריאת הסכמה בלבד (מאוד מהיר!)
schema = pl.read_parquet_schema('../data/venture_funding_deals.parquet')

print("📋 סכמת הקובץ:")
for column_name, data_type in schema.items():
    print(f"  {column_name:<20} {data_type}")

### 💾 כתיבת קבצי Parquet

בואו נשמור DataFrame כקובץ Parquet:

In [ ]:
# המרת CSV ל-Parquet
df_to_save = pl.read_csv('../data/customer_shopping_data.csv')

# שמירה פשוטה
df_to_save.write_parquet('../data/output/shopping_data.parquet')
print("✅ נשמר בהצלחה!")

### 🗜️ בחירת אלגוריתם דחיסה

Parquet תומך במספר אלגוריתמי דחיסה:

In [ ]:
import os

# נשווה גדלי קבצים עם דחיסות שונות
compressions = ['uncompressed', 'snappy', 'gzip', 'lz4', 'zstd']
sizes = {}

for comp in compressions:
    output_path = f'../data/output/test_{comp}.parquet'
    
    # שמירה עם דחיסה ספציפית
    df_to_save.write_parquet(
        output_path,
        compression=comp,
        compression_level=10 if comp != 'uncompressed' else None
    )
    
    # מדידת גודל
    size_mb = os.path.getsize(output_path) / (1024 * 1024)
    sizes[comp] = size_mb

# הצגת תוצאות
print("📊 השוואת גדלי קבצים:")
print("=" * 40)
for comp, size in sorted(sizes.items(), key=lambda x: x[1]):
    print(f"{comp:<15} {size:>8.2f} MB")

#### 🎯 איזה דחיסה לבחור?

| דחיסה | מהירות | גודל | מתי להשתמש |
|-------|---------|------|-------------|
| `uncompressed` | ⚡⚡⚡ | ❌ גדול | בדיקות מהירות |
| `snappy` | ⚡⚡ | ✅ טוב | **ברירת מחדל מומלצת** |
| `lz4` | ⚡⚡ | ✅ טוב | מהיר מאוד |
| `gzip` | ⚡ | ✅✅ קטן | אחסון ארוך טווח |
| `zstd` | ⚡ | ✅✅ קטן | איזון טוב (מומלץ!) |

In [ ]:
# המלצה: zstd עם רמת דחיסה 10
df_to_save.write_parquet(
    '../data/output/shopping_optimized.parquet',
    compression='zstd',
    compression_level=10  # 1-22, 10 = איזון טוב
)
print("✅ נשמר עם דחיסה אופטימלית!")

### ⚡ Lazy Loading ב-Parquet

In [ ]:
# סריקה lazy
lf_parquet = pl.scan_parquet('../data/venture_funding_deals.parquet')

# עיבוד מורכב - הכל lazy!
result = (
    lf_parquet
    .filter(pl.col('Industry') == 'Artificial intelligence')
    .select(['Company', 'Amount', 'Valuation'])
    .head(5)
    .collect()  # רק עכשיו מריץ!
)

print("🤖 חברות AI עם המימון הגבוה ביותר:")
result

### 📁 Partitioned Parquet - קבצים מחולקים

**מה זה Partitioning?**

במקום קובץ אחד ענק, מחלקים את הנתונים לתיקיות לפי ערך:
```
data/
├── Industry=Fintech/
│   └── data.parquet
├── Industry=AI/
│   └── data.parquet
└── Industry=Healthcare/
    └── data.parquet
```

**יתרון**: קריאה מהירה של רק הנתונים הרלוונטיים!

In [ ]:
# קריאת parquet מחולק (צריך pyarrow)
df_partitioned = pl.read_parquet(
    '../data/venture_funding_deals_partitioned',
    use_pyarrow=True,
    pyarrow_options={'partitioning': 'hive'}  # 👈 מצב Hive standard
)

print(f"✅ נטענו {len(df_partitioned):,} שורות")
print(f"תעשיות: {df_partitioned['Industry'].n_unique()}")
df_partitioned.head()

In [ ]:
# כתיבה של parquet מחולק
df_to_partition = pl.read_parquet('../data/venture_funding_deals.parquet')

df_to_partition.write_parquet(
    '../data/output/funding_by_industry',
    use_pyarrow=True,
    pyarrow_options={
        'partition_cols': ['Industry'],  # 👈 חלוקה לפי Industry
        'existing_data_behavior': 'overwrite_or_ignore'
    }
)
print("✅ נוצרה מבנה תיקיות מחולק!")

---

### 🎓 תרגיל 2: Parquet

**משימה:**
1. קרא את `venture_funding_deals.parquet`
2. סנן רק עסקאות מעל $500M
3. שמור כ-parquet עם דחיסת `zstd`
4. השווה את גודל הקובץ המקורי לחדש

In [ ]:
# 💪 נסה בעצמך:


---

### 📚 סיכום: Parquet

**מתי להשתמש ב-Parquet?**
- ✅ אחסון ארוך טווח
- ✅ קבצים גדולים (>100MB)
- ✅ צורך בביצועים
- ✅ עבודה עם ענן

**טיפים:**
- 💾 דחיסה: השתמש ב-`zstd` או `snappy`
- 📂 Partitioning: לנתונים גדולים עם קבוצות לוגיות
- ⚡ Lazy: השתמש ב-`scan_parquet()` לקבצים גדולים

---

## 3. קריאה וכתיבה של טבלאות Delta Lake

### 📖 מה זה Delta Lake?

**Delta Lake** הוא **פורמט אחסון** מתקדם שבנוי על Parquet ומוסיף:
- 🔄 **ACID Transactions** - עסקאות מובטחות
- 📜 **Version History** - היסטוריית שינויים
- ⏮️ **Time Travel** - חזרה לגרסאות קודמות
- 🔍 **Schema Evolution** - שינוי מבנה בטוח

**למי זה מתאים?**
- 🏢 Data Lakes ארגוניים
- 🔄 צנרות נתונים (Data Pipelines)
- 📊 Data Warehouses מודרניים

### 🔍 קריאת Delta Table

In [ ]:
# קריאה בסיסית
df_delta = pl.read_delta('../data/venture_funding_deals_delta')

print(f"📊 נטענו {len(df_delta):,} שורות")
df_delta.head()

### 💾 כתיבת Delta Table

**שלושה מצבי כתיבה:**
- `append` - הוספה לנתונים קיימים
- `overwrite` - מחיקה וכתיבה מחדש
- `error` - שגיאה אם הטבלה קיימת (ברירת מחדל)

In [ ]:
# יצירת Delta Table חדשה
df_to_save = pl.read_csv('../data/customer_shopping_data.csv')

df_to_save.write_delta(
    '../data/output/shopping_delta',
    mode='overwrite'  # 👈 מחיקה והחלפה
)
print("✅ Delta Table נוצרה!")

In [ ]:
# הוספת נתונים חדשים
new_data = pl.DataFrame({
    'invoice_no': ['I999999'],
    'customer_id': ['C999999'],
    'gender': ['Male'],
    'age': [25],
    'category': ['Electronics'],
    'quantity': [1],
    'price': [1500.0],
    'payment_method': ['Credit Card'],
    'invoice_date': ['1/1/2024'],
    'shopping_mall': ['Test Mall']
})

new_data.write_delta(
    '../data/output/shopping_delta',
    mode='append'  # 👈 הוספה בלבד
)
print("✅ נתונים חדשים נוספו!")

### 📁 Partitioned Delta Tables

In [ ]:
# כתיבה עם partitioning
df_funding = pl.read_parquet('../data/venture_funding_deals.parquet')

df_funding.write_delta(
    '../data/output/funding_delta_partitioned',
    mode='overwrite',
    delta_write_options={
        'partition_by': 'Industry'  # 👈 חלוקה לפי תעשייה
    }
)
print("✅ Delta Table מחולקת נוצרה!")

### 🎯 קריאת Partition ספציפי

במקום לטעון הכל, נקרא רק partition אחד:

In [ ]:
# קריאת רק חברות Fintech
df_fintech_only = pl.read_delta(
    '../data/output/funding_delta_partitioned',
    pyarrow_options={
        'partitions': [('Industry', '=', 'Fintech')]  # 👈 סינון!
    }
)

print(f"📊 נטענו רק {len(df_fintech_only):,} שורות מ-Fintech")
df_fintech_only.head()

### ⚡ Lazy Loading ב-Delta

In [ ]:
# סריקה lazy של Delta Table
lf_delta = pl.scan_delta('../data/venture_funding_deals_delta')

result = (
    lf_delta
    .filter(pl.col('Industry') == 'Fintech')
    .select(['Company', 'Amount'])
    .head()
    .collect()
)

result

### ☁️ עבודה עם Delta ב-Cloud (S3, Azure, GCS)

Delta Lake נהדר לעבודה עם אחסון בענן:

In [ ]:
# דוגמה: קריאה מ-S3
# (דורש הגדרת credentials)

storage_options = {
    'aws_access_key_id': 'YOUR_KEY',
    'aws_secret_access_key': 'YOUR_SECRET',
    'aws_region': 'us-east-1'
}

# df = pl.read_delta(
#     's3://my-bucket/my-delta-table',
#     storage_options=storage_options
# )

print("💡 זו דוגמה - דורש הגדרת AWS credentials")

---

### 🎓 תרגיל 3: Delta Lake

**משימה:**
1. צור Delta Table מקובץ CSV
2. הוסף שורות חדשות (append)
3. קרא את הטבלה ובדוק שהשורות התווספו

In [ ]:
# 💪 נסה בעצמך:


---

### 📚 סיכום: Delta Lake

**מתי להשתמש ב-Delta Lake?**
- ✅ צנרות נתונים ייצור
- ✅ צורך ב-ACID transactions
- ✅ Data Lakes ארגוניים
- ✅ עבודה עם ענן

**יתרונות מרכזיים:**
- 🔒 בטיחות: ACID transactions
- 📜 היסטוריה: גרסאות ו-Time Travel
- ⚡ ביצועים: בנוי על Parquet
- 📁 Partitioning: אופטימיזציה אוטומטית

---

## 4. קריאה וכתיבה של קבצי JSON

### 📖 מה זה JSON?

**JSON (JavaScript Object Notation)** הוא פורמט טקסט לאחסון נתונים מובנים.

**שני סוגים:**
1. **JSON** רגיל - מערך של אובייקטים: `[{...}, {...}]`
2. **NDJSON** (Newline Delimited) - אובייקט בכל שורה: `{...}\n{...}\n`

**מתי להשתמש?**
- 🌐 APIs ו-Web Services
- 📦 נתונים מקוננים (nested)
- 🔄 תצורות (configurations)

### 🔍 קריאת JSON רגיל

In [ ]:
# קריאה בסיסית
df_json = pl.read_json('../data/world_population.json')

print(f"📊 נטענו {len(df_json):,} שורות")
print(f"🌍 מדינות: {df_json['country'].n_unique()}")

# הצגת חלק מהעמודות
df_json.select(['country', 'pop2023', 'area', 'density']).head()

### 💾 כתיבת JSON

In [ ]:
# שמירה כ-JSON
df_sample = df_json.head(10)
df_sample.write_json('../data/output/population_sample.json')

print("✅ JSON נשמר!")

# בואו נראה איך הקובץ נראה
with open('../data/output/population_sample.json', 'r') as f:
    print("\n📄 תוכן הקובץ (50 תווים ראשונים):")
    print(f.read()[:200] + "...")

### 📝 NDJSON - JSON שורה אחר שורה

**למה NDJSON?**
- ⚡ מהיר יותר לקבצים גדולים
- 💾 חוסך זיכרון
- 🔄 מתאים ל-streaming

In [ ]:
# קריאת NDJSON
df_ndjson = pl.read_ndjson('../data/world_population.jsonl')

print(f"📊 נטענו {len(df_ndjson):,} שורות מ-NDJSON")
df_ndjson.select(['country', 'pop2023', 'density']).head()

In [ ]:
# כתיבת NDJSON
df_sample.write_ndjson('../data/output/population_sample.jsonl')

print("✅ NDJSON נשמר!")

# הבדל בפורמט:
with open('../data/output/population_sample.jsonl', 'r') as f:
    print("\n📄 NDJSON - כל שורה = אובייקט נפרד:")
    for i, line in enumerate(f):
        if i < 2:  # רק 2 שורות ראשונות
            print(line[:100] + "...")

### ⚡ Lazy Loading ב-NDJSON

רק NDJSON תומך ב-lazy loading!

In [ ]:
# סריקה lazy
lf_ndjson = pl.scan_ndjson('../data/world_population.jsonl')

# מציאת 5 המדינות הצפופות ביותר
result = (
    lf_ndjson
    .select(['country', 'density', 'pop2023'])
    .sort('density', descending=True)
    .head(5)
    .collect()
)

print("🏙️ המדינות הצפופות ביותר:")
result

### 🌳 עבודה עם JSON מקונן

JSON יכול להכיל מבנים מורכבים:

In [ ]:
# דוגמה ל-JSON מקונן
nested_json = '''
[
    {
        "name": "John",
        "age": 30,
        "address": {
            "city": "New York",
            "zip": "10001"
        },
        "hobbies": ["reading", "gaming"]
    },
    {
        "name": "Jane",
        "age": 25,
        "address": {
            "city": "Los Angeles",
            "zip": "90001"
        },
        "hobbies": ["sports", "music", "travel"]
    }
]
'''

# קריאה מ-string
import io
df_nested = pl.read_json(io.StringIO(nested_json))

print("🌳 JSON מקונן:")
df_nested

In [ ]:
# פירוק (unnest) של עמודת struct
df_unnested = df_nested.unnest('address')

print("📦 אחרי unnest:")
df_unnested

---

### 🎓 תרגיל 4: JSON

**משימה:**
1. קרא את `world_population.jsonl`
2. סנן מדינות עם אוכלוסייה מעל 100M
3. שמור כ-NDJSON חדש
4. קרא בחזרה ובדוק

In [ ]:
# 💪 נסה בעצמך:


---

### 📚 סיכום: JSON

| פורמט | Lazy Support | מתי להשתמש |
|--------|--------------|------------|
| JSON | ❌ | קבצים קטנים, APIs |
| NDJSON | ✅ | קבצים גדולים, streaming |

**טיפים:**
- 📦 NDJSON מהיר וחסכוני יותר
- 🌳 השתמש ב-`unnest()` למבנים מקוננים
- ⚡ Lazy: רק NDJSON תומך

---

## 5. קריאה וכתיבה של קבצי Excel

### 📖 מה זה Excel?

קבצי Excel (`.xlsx`, `.xls`) הם פורמט נפוץ בעולם העסקי.

**אתגרים:**
- 📊 יכול להכיל מספר גיליונות (sheets)
- 🎨 עיצוב, נוסחאות, מקרו
- 🐌 איטי יחסית

**הערה:** צריך להתקין תוספים!

In [ ]:
# התקנת חבילות נדרשות (הרץ פעם אחת)
# !pip install xlsx2csv xlsxwriter openpyxl

### 💾 כתיבת קובץ Excel

In [ ]:
# הכנת נתונים
df_to_excel = pl.read_csv('../data/customer_shopping_data.csv').head(100)

# כתיבה בסיסית
df_to_excel.write_excel(
    '../data/output/shopping_data.xlsx',
    worksheet='Sales Data',  # שם הגיליון
    header_format={'bold': True}  # כותרות מודגשות
)

print("✅ קובץ Excel נוצר!")

### 🔍 קריאת קובץ Excel

Excel יכול להכיל מספר גיליונות, אז צריך לציין איזה:

In [ ]:
# קריאה מגיליון ספציפי
df_from_excel = pl.read_excel(
    '../data/output/shopping_data.xlsx',
    sheet_name='Sales Data',  # 👈 שם הגיליון
    engine='xlsx2csv',  # מנוע מהיר
    read_options={'try_parse_dates': True}  # ניתוח תאריכים
)

print(f"📊 נטענו {len(df_from_excel):,} שורות מ-Excel")
df_from_excel.head()

### 📚 עבודה עם מספר גיליונות

In [ ]:
# יצירת workbook עם מספר גיליונות
with pl.ExcelWriter('../data/output/multi_sheet.xlsx') as writer:
    # גיליון 1: כל הנתונים
    df_to_excel.write_excel(
        workbook=writer,
        worksheet='All Data'
    )
    
    # גיליון 2: רק בגדים
    df_clothing = df_to_excel.filter(pl.col('category') == 'Clothing')
    df_clothing.write_excel(
        workbook=writer,
        worksheet='Clothing Only'
    )
    
    # גיליון 3: סטטיסטיקות
    stats = df_to_excel.group_by('category').agg([
        pl.col('price').mean().alias('avg_price'),
        pl.col('quantity').sum().alias('total_quantity')
    ])
    stats.write_excel(
        workbook=writer,
        worksheet='Statistics'
    )

print("✅ Workbook עם 3 גיליונות נוצר!")

### 🎨 עיצוב מתקדם

In [ ]:
# יצירת Excel מעוצב
df_styled = df_to_excel.head(20)

# פורמטים מותאמים אישית
df_styled.write_excel(
    '../data/output/styled_report.xlsx',
    worksheet='Report',
    header_format={
        'bold': True,
        'font_color': 'white',
        'bg_color': '#4472C4',
        'border': 1
    },
    column_formats={
        'price': '₪#,##0.00',  # פורמט מטבע
        'invoice_date': 'dd/mm/yyyy'  # פורמט תאריך
    },
    column_widths={
        'customer_id': 120,
        'shopping_mall': 150
    },
    autofit=True  # התאמה אוטומטית לתוכן
)

print("🎨 דוח מעוצב נוצר!")

---

### ⚠️ הגבלות חשובות

- 📏 Excel מוגבל ל-**1,048,576 שורות**
- 🐌 איטי לקבצים גדולים
- 💾 גודל קובץ גדול

**המלצה:** לנתונים גדולים, השתמש ב-Parquet או CSV!

---

### 🎓 תרגיל 5: Excel

**משימה:** צור דוח Excel עם 3 גיליונות:
1. "Raw Data" - כל הנתונים
2. "Summary" - סיכום לפי קטגוריה
3. "High Value" - רק רכישות מעל 1000

In [ ]:
# 💪 נסה בעצמך:


---

### 📚 סיכום: Excel

**מתי להשתמש:**
- ✅ דוחות עסקיים
- ✅ שיתוף עם משתמשי Excel
- ✅ נתונים קטנים-בינוניים (<100K שורות)

**חסרונות:**
- ❌ איטי
- ❌ מוגבל בגודל
- ❌ לא מתאים לאוטומציה

---

## 9. סיכום וטיפים

### 🎯 טבלת ההחלטות - איזה פורמט לבחור?

| צורך | פורמט מומלץ | למה? |
|------|-------------|------|
| שיתוף עם אנשים | CSV | קל לפתיחה בכל תוכנה |
| דוחות עסקיים | Excel | מוכר ונוח למנהלים |
| אחסון ארוך טווח | Parquet | דחוס, מהיר, שומר טיפוסים |
| Data Lake ארגוני | Delta Lake | ACID, versioning |
| API / Web | JSON/NDJSON | תקן אינטרנט |
| קבצים ענקיים | Parquet + Lazy | אופטימלי לביצועים |
| נתונים זמניים | IPC/Arrow | מהיר ביותר (בינארי) |

---

### ⚡ טיפים לביצועים

#### 1. השתמש ב-Lazy Loading לקבצים גדולים

```python
# ❌ לא יעיל
df = pl.read_csv('huge_file.csv')
df = df.filter(pl.col('age') > 30)

# ✅ יעיל!
result = (
    pl.scan_csv('huge_file.csv')
    .filter(pl.col('age') > 30)
    .collect()
)
```

#### 2. השתמש ב-schema_overrides לחיסכון בזיכרון

```python
# ❌ מבזבז זיכרון
df = pl.read_csv('data.csv')  # age הופך ל-Int64

# ✅ חוסך פי 8!
df = pl.read_csv('data.csv', schema_overrides={'age': pl.Int8})
```

#### 3. בחר את פורמט הדחיסה הנכון

```python
# לאחסון ארוך טווח
df.write_parquet('archive.parquet', compression='zstd', compression_level=10)

# לעבודה יומיומית
df.write_parquet('working.parquet', compression='snappy')
```

#### 4. השתמש ב-Partitioning לנתונים גדולים

```python
# קריאה מהירה של רק הנתונים הרלוונטיים
df.write_parquet(
    'data_partitioned',
    use_pyarrow=True,
    pyarrow_options={'partition_cols': ['year', 'month']}
)
```

---

### 🛠️ פתרון בעיות נפוצות

#### בעיה 1: "Out of Memory"

**פתרון:**
```python
# במקום read_*, השתמש ב-scan_*
lf = pl.scan_parquet('huge.parquet')
result = lf.filter(...).select(...).collect()
```

#### בעיה 2: תאריכים לא מזוהים נכון

**פתרון:**
```python
# אפשר try_parse_dates
df = pl.read_csv('data.csv', try_parse_dates=True)

# או הגדר פורמט ספציפי
df = df.with_columns(
    pl.col('date').str.strptime(pl.Date, '%d/%m/%Y')
)
```

#### בעיה 3: קובץ CSV עם encoding לא תקין

**פתרון:**
```python
# נסה encodings שונים
df = pl.read_csv('data.csv', encoding='utf8-lossy')
# או
df = pl.read_csv('data.csv', encoding='latin1')
```

---

### 📚 משאבים נוספים

- 📖 [Polars Documentation](https://docs.pola.rs/)
- 💬 [Polars Discord](https://discord.gg/4UfP5cfBE7)
- 🐙 [Polars GitHub](https://github.com/pola-rs/polars)
- 📺 [Polars YouTube Channel](https://www.youtube.com/@polarsDataFrame)

---

### 🎉 סיום

**מזל טוב! סיימת את המדריך המקיף לקריאה וכתיבה של קבצים ב-Polars! 🎊**

עכשיו אתה יודע:
- ✅ איך לעבוד עם כל פורמטי הקבצים הנפוצים
- ✅ מתי להשתמש בכל פורמט
- ✅ איך לאופטימייז ביצועים וזיכרון
- ✅ איך לפתור בעיות נפוצות

**המשך ללמוד ולתרגל! 🚀**

---